In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Input, Flatten, Dense
from tensorflow.keras import Model, regularizers

In [3]:


def track_model(input_layer, start_neurons, stop_neurons, l1:float = 0, l2:float = 0):
    
    conv1 = Conv2D(np.floor(start_neurons * 1), (7, 7), activation="relu", padding="same")(input_layer)
    pool1 = MaxPooling2D((2, 2))(conv1)
    
    conv2 = Conv2D(np.floor(start_neurons * 1), (5, 5), activation="relu", padding="same")(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(np.floor(start_neurons * 1), (3, 3), activation="relu", padding="same")(pool2)
    pool3 = MaxPooling2D((2, 2))(conv3)
    

    flat = Conv2D(32, (1,1))(pool3)
    flat = Flatten()(flat)
    
    dense1 = Dense(start_neurons, activation = 'relu')(flat)
    drop = Dropout(0.1)(dense1)

    dense2 = Dense(start_neurons, activation = 'relu')(drop)
    drop2 = Dropout(0.1)(dense2)
    
    output_layer = Dense(stop_neurons, activation = 'linear')(drop2)

    return output_layer


def model_setup(img_size_target, start_neurons:int = 32, stop_neurons:int = 1, l1:float = 0, l2:float = 0):
    inputs = Input((img_size_target, img_size_target, 1))
    return Model(inputs = inputs, outputs = track_model(input_layer=inputs, start_neurons=start_neurons, stop_neurons=stop_neurons, l1=l1, l2=l2), name = "Track_Model")

In [4]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.models import load_model

import tensorboard
from os.path import isfile, join
from random import shuffle

from skimage import io

In [5]:
Data = r'P:\\Data\\Jacopo\\deeptrack_data\\filtered_images\\image_track_1.tiff'
labels = r'P:\\Data\\Jacopo\\deeptrack_data\\tracks\\track_1.h5'

Data = io.imread(Data)

In [6]:
with pd.HDFStore(labels, "r") as hdf:
    labels = np.array([np.array([x, y]) for x, y in zip(hdf["Frame_0"]["x"], hdf["Frame_0"]["y"])])
X_train, X_test, y_train, y_test = train_test_split(Data, labels, test_size=0.2)

In [7]:
X_train = X_train/255
X_test = X_test/255

In [8]:
# Hyperparameters
BATCH_SIZE = 16
BUFFER_SIZE = 1000
EPOCHS = 10

L1 = 1e-6
L2 = 1e-5
LR = 0.0001
decay_rate = 3*LR/EPOCHS

In [9]:
model = model_setup(255,16,2,1e-5,1e-5)

In [10]:
model.summary()

Model: "Track_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 255, 255, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 255, 255, 16)      800       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 127, 127, 16)      6416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 16)        

In [11]:
# Other Params
opt = Adam(learning_rate=LR, decay = decay_rate, clipnorm = 1)#, clipnorm = 1

loss = MeanSquaredError()

In [12]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [13]:
X_train.shape

(3940, 255, 255)

In [14]:
History = model.fit(X_train, y_train, epochs=EPOCHS, validation_split=0.2, batch_size=1)

Epoch 1/10
3152/3152 [==============================] - 45s 13ms/step - loss: 6754.0029 - accuracy: 0.5574 - val_loss: 5268.4468 - val_accuracy: 0.5952
Epoch 2/10
3152/3152 [==============================] - 44s 14ms/step - loss: 5873.1772 - accuracy: 0.5622 - val_loss: 5117.2803 - val_accuracy: 0.5952
Epoch 3/10
2044/3152 [==================>...........] - ETA: 24s - loss: 5778.1602 - accuracy: 0.5597